In [46]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from _config import HUFFINGFACE_KEY

##NNSIGHT

import nnsight
from nnsight import NNsight, LanguageModel


##TRANSFORMER-LENS

# import transformer_lens.utils as utils
# from transformer_lens.hook_points import (
#     HookPoint,
# )  # Hooking utilities
from transformer_lens import HookedTransformer

In [47]:
prompt = 'The Eiffel Tower is in the city of'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
# Transformer-lens
model_name = "gpt2"
# model = AutoModelForCausalLM.from_pretrained(model_name, force_download=True)
# tokenizer = AutoTokenizer.from_pretrained(model_name, force_download=True)
model1 = HookedTransformer.from_pretrained('gpt2', device=device)


Loaded pretrained model gpt2 into HookedTransformer


In [60]:
model1.generate(prompt, max_new_tokens=1,prepend_bos=True)

100%|██████████| 1/1 [00:00<00:00, 11.75it/s]


'The Eiffel Tower is in the city of Paris'

In [61]:
2## NNsight
# model_name = "gpt2"
# model = AutoModelForCausalLM.from_pretrained(model_name, force_download=True)
# tokenizer = AutoTokenizer.from_pretrained(model_name, force_download=True)
model2 = LanguageModel('gpt2', device_map='auto')
with model2.trace(prompt):
    output = model2.lm_head.output.save()

model1.tokenizer.decode(torch.argmax(output, dim=-1)[0])

'\n-el Tower is a the middle centre Paris'

# Section 1

In [33]:
# Standard
model_name = 'gpt2'
batchsize = 4
quantize = ''
data = 'DC'
trial = True
method = 'logit-lens'
cache = os.path.expanduser('./tmp/cashe/')

# article2tokens = json.load(open(f"data/{data}/tokens.json"))
# loss_fct = CrossEntropyLoss(ignore_index=-100, reduction="none")

access_token = HUFFINGFACE_KEY
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache)
tokenizer.pad_token = tokenizer.eos_token

gpt2_model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token, cache_dir=cache)
gpt2_model.to(device).eval()

tokenized = tokenizer(prompt, return_tensors="pt", padding=True, add_special_tokens=False)["input_ids"].to(device)
output = gpt2_model(tokenized, output_hidden_states=True)
logit = gpt2_model.lm_head(torch.stack(output[2]).to(device))
tokenizer.decode(torch.argmax(logit, dim=-1)[0][-1])

RecursionError: maximum recursion depth exceeded

In [6]:
# Tokenize the prompt
model2 = LanguageModel('gpt2', device_map='auto')
input_ids = model2.tokenizer(prompt, return_tensors='pt').input_ids.to(model2.device)

# Store generated tokens
generated = input_ids.clone()

# Autoregressive generation loop
for _ in range(10):
    with model2.trace(generated) as trace:
        output = trace.model.lm_head.output.save()

    logits = output.value
    next_token_logits = logits[:, -1, :]
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)

    generated = torch.cat((generated, next_token_id), dim=1)

# Decode full output
output_text = model2.tokenizer.decode(generated[0], skip_special_tokens=True)
print(output_text)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


NotImplementedError: Cannot copy out of meta tensor; no data!

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
# it seems that all packages actually acess the same cache somewhere, force_downloading the model using transformers seem to improve all of them